<center> <h1> <b>Activity-dependent self-organisation</b> in an E/I network with <b>homeostatic structural plasticity</b></h1>
<a href="mailto:aadhar@pm.me">Aadhar Sharma</a>  <br>
 Bernstein Center Freiburg<br>
 University of Freiburg<br>
 Germany
 </center>
<hr>

## Theory:
<font size="5">1. E/I network <br></font>
    
<center>
    <img src="./assets/base_network.png" alt="E/I network with plastic recurrent connections" class="bg-primary mb-1" width="500px">
</center>
    

<font size="5">2. Homeostatic structural plasticity<br></font>


<center>
    <img src="./assets/SP_connect_DEF.svg" alt="synaptic elements" class="bg-primary mb-1" width="500px"><br>
    <!-- <img src="./assets/SP_algo.png" alt="SP_algo" class="bg-primary mb-1" width="600px"><br><br><br> -->
    <br><br><br>
    <img src="./assets/SP_equilibrium.svg" alt="frh_demo" class="bg-primary mb-1" width="700px"><br><br>
    <img src="./assets/SP_delete.svg" alt="frh_demo" class="bg-primary mb-1" width="700px"><br><br>
    <img src="./assets/SP_create.svg" alt="frh_demo" class="bg-primary mb-1" width="700px">
</center>
<br>
<hr>
For more on the homeostatic structural plasticity model see: <a href="https://www.frontiersin.org/articles/10.3389/fnana.2016.00057/full">Diaz-Pier, S. et al. Frontiers in Neuroanatomy. doi:10.3389/fnana.2016.00057 (2016).</a>
<hr>

## Tutorial
### 1. Importing necessary modules

In [ ]:
import nest # v.3.4
import numpy as np
from time import time
from IPython.display import clear_output
import matplotlib as mpl    
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec as GC
from scipy import sparse as sp

# [optional]
import seaborn as sbn
sbn.set()

In [ ]:
## Helper functions for plotting

#### IGNORE THEM NOW, PLAY WITH THEM LATER (after the tutorial) ####


def cosmetics(isPrint=True, isPlot=True):
    """
    Plot acitivty and connectivity dynamics for each simulation step
    """
    
    SR_to_rates = (lambda ids: np.unique(ids, return_counts=True)[1]/growth_step_dur*1e3)  # firing rate in Hz
    
    # clear_output(wait=True)
    tax = simulation_time_axis/1e3  # [s]
    SP_onset_time = tax[SP_enable_index-1]
    
    fig = plt.figure(layout="constrained", figsize=np.array([12, 5])*2)
    gs = GC(2, 10, figure=fig, hspace=0.05, wspace=0.18)
    ms = 3.5
    
    ax_activity = fig.add_subplot(gs[0, 0:3]) 
    ax_calcium = fig.add_subplot(gs[1, 0:3])
    ax_Z = fig.add_subplot(gs[0, 3:6])
    ax_raster = fig.add_subplot(gs[:, 6:])
    ax_Connectivity = fig.add_subplot(gs[1, 3:6])

    
    senders_exc, times_exc = (spike_recorder_exc.events[label] for label in ["senders", "times"])
    senders_inh, times_inh = (spike_recorder_inh.events[label] for label in ["senders", "times"])
    activity_exc[ii, np.unique(spike_recorder_exc.events["senders"])-1] = SR_to_rates(spike_recorder_exc.events["senders"])
    activity_inh[ii, np.unique(spike_recorder_inh.events["senders"])-1-N_exc] = SR_to_rates(spike_recorder_inh.events["senders"])

    plt.sca(ax_activity)
    plt.axhline(Ca_setpoint*1e3, ls="--", color="darkred", lw=1, label="target rate")
    plt.axvline(SP_onset_time-1, ls="--", color="orange", label="SP onset")
    for activity, c, lab in zip([activity_exc, activity_inh], 
                                ["r", "g"],
                                ["Exc.", "Inh."]):
        mu = np.nanmean(activity, 1)
        std = np.nanstd(activity, 1)
        plt.plot(tax, mu, color=c, label=lab, lw=1.25, ls="-", marker="o", markersize=ms)
        plt.fill_between(tax, mu-std, mu+std, color=c, alpha=0.25)
    plt.legend()
    plt.ylabel("firing rate [Hz]")
    plt.title("Activity")

    plt.sca(ax_calcium)
    Ca_exc[ii] = neuronal_pop_exc.get("Ca")
    mu = np.nanmean(Ca_exc, 1)
    std = np.nanstd(Ca_exc, 1)
    
    plt.axvline(SP_onset_time-1, ls="--", color="orange", label="SP onset")
    plt.axhline(Ca_setpoint, ls="--", color="darkred", label=r"$Ca^*$")
    plt.plot(tax, mu, color="red", label="Exc.", lw=1.25, ls="-", marker="o", markersize=ms)
    plt.fill_between(tax, mu-std, mu+std, color="r", alpha=0.25)
    plt.xlabel("time [s]")
    plt.ylabel("$Ca [a.u.]")
    plt.title("Cytosolic calcium conc.")
    plt.legend()
    
    plt.sca(ax_raster)
    plt.scatter(times_exc/1e3, senders_exc, marker=".", color="r", s=1)
    plt.scatter(times_inh/1e3, senders_inh, marker=".", color="g", s=1)
    plt.title(f"{sim_step/1e3} [s]")
    plt.ylabel("neuron id")
    plt.xlabel("simulation time [s]") 

    
    # extract data of interest
    Konns = nest.GetConnections(neuronal_pop_exc+neuronal_pop_inh, neuronal_pop_exc+neuronal_pop_inh)
    if not len(Konns)==0:
        sources = Konns.source
        targets = Konns.target
        # weight = Konns.weight
        AdjMat_timestep = sp.coo_matrix((np.ones(len(sources)), (targets, sources)))
        connectivity_EE[ii, :] = AdjMat_timestep.todok()[0:N_exc, 0:N_exc].mean(1).squeeze()
        AdjMat_max = AdjMat_timestep.max()

    plt.sca(ax_Connectivity)
    plt.axvline(SP_onset_time-1, ls="--", color="orange", label="SP onset")
    mu = np.nanmean(connectivity_EE, 1)*1e2
    std = np.nanstd(connectivity_EE, 1)
    plt.plot(tax, mu, color="red", label=r"$Exc. \longrightarrow Exc.$", lw=1.25, ls="-", marker="o", markersize=ms)
    plt.fill_between(tax, mu-std, mu+std, color=c, alpha=0.25)
    # plt.ylim([-, ax_Connectivity.get_ylim()[1]])
    plt.title("Connectivity")
    plt.ylabel("percent connectivity")
    plt.xlabel("time [s]")
    plt.legend()
    

    plt.sca(ax_Z)
    Z = neuronal_pop_exc.get("synaptic_elements")
    Z_dendritic[ii]=[ZZ["dendritic_element"]["z"] for ZZ in Z]
    Z_connected[ii] = [ZZ["dendritic_element"]["z_connected"] for ZZ in Z]

    plt.plot(tax, np.nanmean(Z_dendritic, 1), "-", color="fuchsia", label="all", marker="o", markersize=ms)
    plt.plot(tax, np.nanmean(Z_connected, 1), "--", color="firebrick", label="conn.", marker="o", markersize=ms)

    plt.ylim([-10, ax_Z.get_ylim()[1]])
    plt.axvline(SP_onset_time-1, ls="--", color="orange", label="SP onset")
    plt.ylabel("no. of dendritic elements per neuron")
    plt.title("Synaptic elements")
    plt.legend()


    if not ii < SP_enable_index:
        xticks = np.linspace(tax[0], tax[ii], 10)
        for ax in [ax_activity, ax_calcium, ax_Z, ax_Connectivity]:
            ax.set_xticks(xticks)
            
    long_text = f"iter={ii+1}/{simulation_time_axis.size},"+"    "+\
                f"#K={len(Konns)},"+"    "+\
                f"E[r]={np.nanmean(activity_exc[ii]).round(3)} [Hz],"+"    "+\
                f"Ca*={Ca_setpoint},"+"    "+\
                f"E[Ca]={np.mean(neuronal_pop_exc.get('Ca')).round(5)},"+"    "+\
                f" Z_d={np.mean([ x['dendritic_element']['z'] for x in neuronal_pop_exc.get('synaptic_elements')]).astype('int')}"+"\n"
    
    if isPlot:
        plt.suptitle(long_text, fontsize=16)  
        plt.show()

    if isPrint:
        print(f"iter time: {iter_dur} [s]")
        Konns = nest.GetConnections(neuronal_pop_exc, neuronal_pop_exc)
        if not isPlot:
            print(long_text)  
        print("----------------------------------------------------------------------------------")


def adj_end(isPlot=True):
    """
    Plot Connectivity Matrices at the beginning and end of the simulation
    """
    
    if isPlot:
        f, ax = plt.subplots(1, 2, figsize=[18, 8])
        ax_init = ax[0]
        ax_adjmat = ax[1]


        Konns =  nest.GetConnections(neuronal_pop_exc+neuronal_pop_inh, neuronal_pop_exc+neuronal_pop_inh, "static_synapse")
        sources = Konns.source
        targets = Konns.target
        AdjMat_timestep = sp.coo_matrix((np.ones(len(sources)), (targets, sources)))
        connectivity_EE[ii, :] = AdjMat_timestep.todok()[0:N_exc, 0:N_exc].mean(1).squeeze()
        AdjMat_max = AdjMat_timestep.max()
        
        plt.sca(ax_init)
        plt.imshow(AdjMat_timestep.toarray(), cmap="CMRmap_r", vmin=0, vmax=AdjMat_max, interpolation="nearest", origin="lower")   
        ax_init.add_patch(mpl.patches.Rectangle((-1, -1), N_exc-1, N_exc-1, facecolor=[0, 0, 0, 0], edgecolor="r", lw=3, ls="--"))
        ax_init.add_patch(mpl.patches.Rectangle((N_exc+2, N_exc+2), N_inh-1, N_inh-1, facecolor=[0, 0, 0, 0], edgecolor="k", lw=1, ls="--"))
        ax_init.add_patch(mpl.patches.Rectangle((-1, N_exc+1), N_exc, N_inh+1, facecolor=[0, 0, 0, 0], edgecolor="k", lw=1, ls="--"))
        ax_init.add_patch(mpl.patches.Rectangle((N_exc+1, -1), N_inh+1, N_exc-1, facecolor=[0, 0, 0, 0], edgecolor="k", lw=1, ls="--"))
        cmap = mpl.cm.CMRmap_r
        norm = mpl.colors.BoundaryNorm(np.arange(0, AdjMat_max+1), cmap.N)
        cb = plt.colorbar(mpl.cm.ScalarMappable(cmap=cmap, norm=norm), ax=ax_init,
                          use_gridspec=True, shrink=0.75, label="multiplicity", drawedges=True)
        cb.set_ticks(np.arange(0, AdjMat_max)+0.5)
        cb.set_ticklabels(np.arange(0, AdjMat_max).astype("int"))
        plt.title("Initial Connectivity matrix"+"\n"+f"(sim time: {0} [m])")

        
        Konns = nest.GetConnections(neuronal_pop_exc+neuronal_pop_inh, neuronal_pop_exc+neuronal_pop_inh)
        sources = Konns.source
        targets = Konns.target
        
        AdjMat_timestep = sp.coo_matrix((np.ones(len(sources)), (targets, sources)))
        connectivity_EE[ii, :] = AdjMat_timestep.todok()[0:N_exc, 0:N_exc].mean(1).squeeze()
        AdjMat_max = AdjMat_timestep.max()

        plt.sca(ax_adjmat)
        plt.imshow(AdjMat_timestep.toarray(), cmap="CMRmap_r", vmin=0, vmax=AdjMat_max, interpolation="nearest", origin="lower")   
        ax_adjmat.add_patch(mpl.patches.Rectangle((-1, -1), N_exc-1, N_exc-1, facecolor=[0, 0, 0, 0], edgecolor="r", lw=3, ls="--"))
        ax_adjmat.add_patch(mpl.patches.Rectangle((N_exc+2, N_exc+2), N_inh-1, N_inh-1, facecolor=[0, 0, 0, 0], edgecolor="k", lw=1, ls="--"))
        ax_adjmat.add_patch(mpl.patches.Rectangle((-1, N_exc+1), N_exc, N_inh+1, facecolor=[0, 0, 0, 0], edgecolor="k", lw=1, ls="--"))
        ax_adjmat.add_patch(mpl.patches.Rectangle((N_exc+1, -1), N_inh+1, N_exc-1, facecolor=[0, 0, 0, 0], edgecolor="k", lw=1, ls="--"))
        cmap = mpl.cm.CMRmap_r
        norm = mpl.colors.BoundaryNorm(np.arange(0, AdjMat_max+1), cmap.N)
        cb = plt.colorbar(mpl.cm.ScalarMappable(cmap=cmap, norm=norm), ax=ax_adjmat,
                          use_gridspec=True, shrink=0.75, label="multiplicity", drawedges=True)
        cb.set_ticks(np.arange(0, AdjMat_max)+0.5)
        cb.set_ticklabels(np.arange(0, AdjMat_max).astype("int"))
        plt.title("Final Connectivity matrix"+"\n"+f"(sim time: {growth_dur/1e3/60} [m])")

        for axx in ax:
            plt.sca(axx)
            plt.xlim([-10, N_total+10])
            plt.ylim([-10, N_total+10])
    
            plt.xticks([N_exc/2, N_exc+N_inh/2], labels=["Exc.", "Inh."])
            plt.yticks([N_exc/2, N_exc+N_inh/2], labels=["Exc.", "Inh."])

        plt.show()

def plot_SP_model():
    f = plt.figure(figsize=[10, 4])
    x = np.linspace(0, Ca_setpoint*2, 50)
    y = growth_rate*(1-x/Ca_setpoint)
    plt.axvline(Ca_setpoint, color="k", ls="--", label=r"$Ca^*$")
    plt.axhline(0, color="k", lw=0.5, alpha=0.5)
    plt.plot(x, y, c="blue", lw=2)
    plt.xlabel("Ca(t) [a.u.]")
    plt.ylabel(r"$\frac{d}{dt}z(t)$")
    tks = plt.gca().get_yticks()
    plt.yticks([tks[2], 0, tks[-3]], labels=["negative", "FRH: 0", "positive"])
    plt.legend()
    plt.title("Homeostatic structural plasticity: linear rule"+"\n\n"+r"$\frac{d}{dt}z(t)=\rho(1-\frac{Ca(t)}{Ca^*})$")
    plt.show()

---
### 2. Perpare Nest Kernel

In [ ]:
nest.ResetKernel()  # reset NEST Kernel 
nest.set_verbosity("M_ERROR")  # supress verbose output from NEST kernel
nest.rng_seed = np.random.default_rng().integers(low=1, high=2**31)  #  set a random seed between [1, 2^31] | ensures that all simlations runs are unqiue

---
### 3. Initialise parameters for neuron model and network

#### 3.1 Set neuronal parameters

In [ ]:
# neuronal parameters
neuron_model = "iaf_psc_delta"
custom_params = {"tau_m":25,  # [ms]
                }
custom_neuron_model = "custom_neuron_model"
nest.CopyModel(existing=neuron_model, new=custom_neuron_model, params=custom_params)

tau_Ca = 1e4  # [ms] [Ca++]i dynamics time-constant
beta_Ca = 1/tau_Ca  # differs from model default; default = 0.001

# all customised parameters can be bundled into a parameter dictionary which is used to initialise neurons with custom param values
# note that, for sake of simplicity, we only want to customise the excitatory neurons in our E/I network
param_dict_exc = {"tau_Ca":tau_Ca,
                  "beta_Ca":beta_Ca
                 }  

#### 3.2 Set network size (no. of neurons)

In [ ]:
# network parameters
N_exc = int(1.75e3)
N_inh = int(N_exc/4)
N_total = N_exc + N_inh

#### 3.3 Set properties for excitatory and inhibitory synapses
- set synapse properties such that the E/I network is balanced

In [ ]:
# synaptic parameters
J = 0.1 # mV
J_exc = 0.1
g = 8
J_inh = -g*J

#### 3.4 Set static connectivity parameters
- Static connectivity is established at random.
- <font size=5>$\epsilon$</font>$\%$ of neurons in a pre-synaptic population are randomly connected to each post-synaptic neuron
    - [fixed_indegree](https://nest-simulator.readthedocs.io/en/v3.0/guides/connection_management.html#fixed-indegree): "The nodes in pre are randomly connected with the nodes in post such that each node in post has a fixed indegree."

In [ ]:
eps = 10e-2  # 10% 
C_exc = int(eps*N_exc)  # fixed indegree for excitatory neurons
C_inh = int(eps*N_inh)  # fixed indegree for inhibitory neurons

#### 3.5 Set parameters for the external stimulator
- This externel stimulator models the background noise that neurons get in an *in vivo* scenario
    - cortical neuron get input from upto 10,000 neurons
- Background drive is important to kick start the activity in a network
    - sans drive, the network will be completely silent
- Background drive is well approximated by Poisson noise (external stimulator: `poisson_generator`)

In [ ]:
# external stimulator parameters
stimulator_model = "poisson_generator"

# use theoretical formula to set background rate for the network 
E_L, V_th,  tau_m = (nest.GetDefaults(custom_neuron_model)[key] for key in ["E_L", "V_th", "tau_m"])
theta = abs(V_th - E_L)
stimulation_rate_th = theta/(C_exc*J*tau_m)*1e3*C_exc  # [see: https://link.springer.com/article/10.1023/A:1008925309027 ]
stimulation_rate = stimulation_rate_th*2.5
print(f"current background drive is set at {stimulation_rate} [Hz]")

#### 3.6 Setting up structural plasticity model
- The homeostatic structural plasticity model regulates neuronal connectivity by creating or deleting synaptic elements ($z$)
- It is assumed that neurons have a set point for internal calcium concentration ($Ca^*$)
- If the current calcium concetration of the neuron ($Ca(t)$) is greater (or smaller) than the target ($Ca^*$), then neuronal connectivity is down- (or up-regulated) by destroying (or creating) synaptic elements.
    - One simple rule for the dynamics of synaptic elements ($\frac{d}{dt}z(t)$) is the linear rule: $\frac{d}{dt}z(t)=\rho(1-\frac{Ca(t)}{Ca^*})$)
    - Other rules (eg. Gaussian) also exist: see [Diaz-Pier et al](https://www.frontiersin.org/articles/10.3389/fnana.2016.00057/full) and [SP example](https://nest-simulator.readthedocs.io/en/v3.0/auto_examples/structural_plasticity.html).
- It is assumed that $Ca(t)$ is a low-pass version of neuronal activity. (think about Calcium imaging)
    - Therefore, for a cytosolic calcium target ($Ca^*$), there is an equivalent firing rate target
- Once the structural plasticity homeostat establishes the target calcium level, neuronal firing rate is also stablised at the respective firing target.
    - This is called **firing rate homeostatis (FRH)**
- Homeostatic structural plasticity operates at slow time-scales. A few orders of magnitude slower than neuronal dynamics.

In [ ]:
# define a firing rate target
FRH_target = 8  # [Hz]

# The firing rate target is converted to Calcium target as follows
Ca_setpoint = FRH_target*1e-3 

# homeostatic structural plasticity parameters
growth_curve = "linear"
z0 = 0  # no. of synaptic elements to start with at the begining of the simulation. t = 0
growth_rate = Ca_setpoint*0.35  # the rate of linear growth/decay | the rho parameter in the equation above

tau_vacant = 10e-2 # the percentage of the z_vacant that will be deleted on next SP update.
                   # vacant synaptic elements are those wich have no partners. That is ones that are not connected to their respective pre- or post-elements.


SP_update_interval = 250.0  # [ms] | cf. simulation resolution = 0.1 ms

################## TASK #####################
# plot the linear synaptic plasticity model just to visualise it.
# You know that dz/dt = rho*(1-Ca(t)/Ca*)
# Say, x is an array of Ca(t) values. For example, x = np.arange(0, Ca_setpoint*2, 1e-3)
# make a plot (x, y) such that x is Ca(t) and y is dz/dt
# give appropriate axes to your plot.
# .
# .
# .
# or simply, call: plot_SP_model(). But please, give it a try first!
#______________ TASK SPACE BELOW ______________#
# take as much space as you need.



#______________ TASK SPACE OVER ______________#



# put SP parameters in a dictionary. The dictionary keys are **keywords** for NEST's implementation of SP
SP_dict_pre ={"growth_curve":growth_curve,
              "z":z0,
              "growth_rate":growth_rate,
              "eps":Ca_setpoint,
              "continuous":False,
              "tau_vacant":tau_vacant}

SP_dict_post = SP_dict_pre.copy()  # for simplicity, provide same dynamics to post-synaptic elements

synaptic_elemnets_properties = {"axonal_element":SP_dict_pre,
                                "dendritic_element":SP_dict_post}  # the dendritic and axonal elements will have the same dynamics. One can, of course, change this. 
                                                                   # But, be aware that this affects network dynamics!!
                                                                   # The keys to this dict are not NEST keywords, and therefore can be arbitrary. 
                                                                   # However, these key string will be required when specifying the synapse (next section). 
                                                                   # * It's convenient (and, in our case, good practice) to define the keys as string variables. Can you do that? [optional task]. 
                                                                   # * Don't forget to make the appropriate updates to the next section!


# Add these synaptic elements to the parameter dictionary for exc. neurons that we defined in section 3.1
    # Remember, synaptic elements are associated with neurons. And their dynamics depend on the cytosolic calcium levels of these neurons.
    # in our case, we are providing excitatory neurons with structural plasticity. Therefore, we add these synaptic elements' properties to the param_dict_exc only.
param_dict_exc.update({"synaptic_elements":synaptic_elemnets_properties})

#### 3.7 define the structurally plastic synapse 
- based on the parameters you've initiliased in the previous section, tell nest to make a new structurally plastic synapse model
- NEST needs to be told that **your** SP synapse can only be created when the defined free axonal-element element matches with a free dendritic-element.

In [ ]:
# One has to now define the SP synapse model
SP_synapse_model="SP_custom_synapse"  # call it anything. No seriously, go ahead. Call it Hector or Shabnam or THE_CNS*23_synapse

# - we will use the 'static_synapse' model from NEST
# - give it a new name (eg. `SP_custom_synapse`)
# - and, change its default parameters (eg. "weight")
nest.CopyModel(existing="static_synapse", new=SP_synapse_model,
               params={"weight":J_exc}
              )  # creates a new SP synapse

# tell NEST the about the pre- and post- parts of *your* SP synapse model
nest.structural_plasticity_synapses = {"exc-exc":{"synapse_model":SP_synapse_model,
                                                  "pre_synaptic_element": "axonal_element",
                                                  "post_synaptic_element": "dendritic_element"}}  # Adds SP_synapse_model to the list of structurally plastic synapses in NEST kernel
                                                                                                  #     - The name of this SP synapse model in *NEST's SP synapses list* will be `exc-exc`. 
                                                                                                  #     - However, you don't need to worry about that. For you, this is *your* synapse. That is, `SP_synapse_model`
# * The "synapse_model", "pre_synaptic_element", and "post_synaptic_elements" are NEST keywords
# * The string arguments that follow it must match the keys of `synaptic_elements_properties` dict. defined in the previous section

# set SP update interval
nest.structural_plasticity_update_interval =  SP_update_interval

---
### 4. Create neurons and devices, and connect them to form the network

1. Create neurons:
    - Exc. population with `N_exc` neurons and params (`param_dict_exc`)
    - Inh. pop. with `N_inh` neurons<br><br>
2. Create devices:
    - external stimulator (PG)
    - spike recorders (SG)<br><br>
3. Establish all connections
    - PG to Exc. and Inh.
    - Exc. to Inh.
    - Inh. to Exc. and Inh.
    - Exc. to SR_Exc.
    - Inh. to SR_Inh.
  
<center>
    <img src="./assets/base_network_fully_connected.png" alt="E/I network with plastic recurrent connections" class="bg-primary mb-1" width="500px"><br>
    A schematic of the network model 
</center>

In [ ]:
# 1. create neurons
neuronal_pop_exc = nest.Create(neuron_model, N_exc, param_dict_exc)
neuronal_pop_inh = nest.Create(neuron_model, N_inh)

# 2. Create  devices
# 2.1 create external stimulator
stimulator = nest.Create(stimulator_model, 1, {"rate":stimulation_rate})

# 2.2 Create Spike Recorders
spike_recorder_exc, spike_recorder_inh = nest.Create("spike_recorder", 2)

# 3. Establish all Connections
# 3.1 Connect stimulator to neurons
nest.Connect(pre=stimulator, post=neuronal_pop_exc+neuronal_pop_inh, 
             syn_spec={"weight":J})

# 3.2 Connect neurons (establish static connectivity)
nest.Connect(pre=neuronal_pop_exc, post=neuronal_pop_inh, 
             conn_spec={"rule":"fixed_indegree", "indegree":C_exc},
             syn_spec={"weight":J_exc})

nest.Connect(pre=neuronal_pop_inh, post=neuronal_pop_exc+neuronal_pop_inh,
             conn_spec={"rule":"fixed_indegree", "indegree":C_inh},
             syn_spec={"weight":J_inh})


###########  TASK ###########
# 3.3 Connect spike recorders 
nest.Connect(neuronal_pop_exc, spike_recorder_exc)
nest.Connect(neuronal_pop_inh, spike_recorder_inh)

---
### 5. Set simulation in a stepwise manner
- To observe the dynamics we will run the simulation in a stepwise manner
    - total simulation duration = (simulation cycles)*(simulation step size)
- Pre-allocate arrays for data to plot

In [ ]:
# simulation grown parameters
growth_step_dur = 5*1e3  # [ms] i.e., 5 seconds | duration of one simulation step
simulation_cycles = 35  # number of simulation steps
growth_dur = simulation_cycles * growth_step_dur  # total duration of simulation

# set up a simulation time axis
simulation_time_axis = np.arange(start=0, stop=growth_dur + growth_step_dur, step=growth_step_dur)

# pre-allocate arrays for data of interst
Z_connected = np.empty([simulation_time_axis.size, N_exc])*np.nan
Z_dendritic = np.empty([simulation_time_axis.size, N_exc])*np.nan
connectivity_EE = np.empty([simulation_time_axis.size, N_exc])*np.nan
activity_exc = np.empty([simulation_time_axis.size, N_exc])*np.nan
activity_inh = np.empty([simulation_time_axis.size, N_inh])*np.nan
Ca_exc = np.empty([simulation_time_axis.size, N_exc])*np.nan

---
### 6. Simulate
#### 6.1 Disable Structural plasticity temporarily to compare the two behaviours
<center>
    <img src="./assets/base_network_preSP.png" alt="E/I network without SP" class="bg-primary mb-1" width="500px">
    <img src="./assets/base_network_fully_connected.png" alt="E/I network with SP" class="bg-primary mb-1" width="500px">
</center>
    

In [ ]:
nest.DisableStructuralPlasticity()  # Disables structural palsticity
SP_enable_index = 11  ## enable structural plasticty on this index

#### 6.2 execute stepwise simulation (finally!)
- The simulation should take around 15 mins. to finish.
    -  So sit back and relax, or take a break.
- **Question time!**
    -  Have problems? Don't quite get something? Stuck?
        -   This is the time!
  

In [ ]:
time_overall_start = time()  # [optional]: Time overall simulation 

for ii, sim_step in zip(range(simulation_time_axis.size), 
                        simulation_time_axis):
                            
    if ii == SP_enable_index:  # check if SP should be enabled at the current iteration
        nest.EnableStructuralPlasticity()  # enable structural plasticity
        print("------------------------- SP is enabled -------------------------")
        
    # reset recorders
        # recorders need to be reset for each simulation step otherwise they will hold data from previous simulation steps            
    for recorder in (spike_recorder_exc, spike_recorder_inh):
        recorder.n_events=0  # set number of recorded events to 0; effectively resets the recorder.
    
    # perform simulation step
    time_sim_step_start = time()  # [optional]: Time simulation step
                            
    nest.Simulate(growth_step_dur)  # call to simulate for the duration of one simualtion step
                            
    time_sim_step_end = time()
    iter_dur = time_sim_step_end-time_sim_step_start

    # display simulation data for each step
    cosmetics(isPrint=True, isPlot=True)

    # clear output space for future plots
    if not ii == simulation_time_axis.size:
        clear_output(wait=True)

time_overall_end = time()
overall_dur = time_overall_end-time_overall_start
print(f"simulation duration (realworld time)={overall_dur/60} [mins.]")

#### 6.3 Show results at the end of the simulation

In [ ]:
# The following functions were defined earlier. Feel free to play around and break them. But please do so after the tutorial.
cosmetics(isPrint=True, isPlot=True)  # plots the activity and connectivity dynamics
adj_end(isPlot=True)  # plots the connectivity matrices without (simulation begin) and with (simulation end) structurally plastic connections

### Finish.
---

- Done? **Kudos!** Here is a post-tutorial exercise:
    -  Retry the simulations at your own pace (post-tutorial) and **check if the following parameters lead to expected behaviour**. Ask yourself what can I expect when I increase or decrease this parameter? Do the simulations agree with your expectations? If not, why not?
        1.  Target firing rates (or target calcium concentration)
        2.  Background drive to neurons.
        3.  Growth rate of synaptic elements.
        4.  Distinct dynamics of pre- and post-synaptic elements.
        5.  Growth curve of synaptic elements (you're now experts in the linear growth role. It is time to try the Gaussian growth curve. See. <a href="https://www.frontiersin.org/articles/10.3389/fnana.2016.00057/full">Diaz-Pier, S. et al. Frontiers in Neuroanatomy. (2016).</a>)
        6.  Number of neurons
 
      
## How can you use homeostatic structural plasticity in your projects?
- Storing memory engrams<br>
        - <a href="https://www.nature.com/articles/s41598-018-22077-3">Gallinaro, J.V., Rotter, S. <b>Associative properties of structural plasticity based on firing rate homeostasis in recurrent neuronal networks.</b> Sci Rep 8, 3754 (2018)</a><br>
        - <a href="https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1009836"> Gallinaro JV, Gašparović N, Rotter S. <b>Homeostatic control of synaptic rewiring in recurrent networks induces the formation of stable memory engrams.</b> PLOS Computational Biology (2022)</a><br><br>
- <b>Case study: Modelling Adult Neurogenesis</b> <i>(shameless plug)</i> <br>
        - <a href="https://cns2023.sched.com/event/1NYY0/p177-the-dynamics-of-adult-neurogenesis-in-the-dentate-gyrus">CNS*23 <b>Poster #P177: The Dynamics of Adult Neurogenesis in the Dentate Gyrus</b> (Tuesday, July 18 • 16:50 - 18:50)</a><br><br>
- Have something specific in mind? But still don't know how to start? Don't hesitate to contact. :)
        
        
        


